**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Wed Feb 22 07:44:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    27W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
'133- PARTHIBAN M - Javascript-Assignmnet-3.docx'
'190701129_cn exp3.gdoc'
'190701129_cn exp3.pdf'
'190701129_cn_ exp4.gdoc'
'190701129_cn_ exp4.pdf'
 AJAX.gdoc
 appsheet
'ART OF DIPLOMACY.gdoc'
'Assignment 4 (1).docx'
'Assignment 4.docx'
 AWS_Academy_Graduate___AWS_Academy_Cloud_Foundations_Badge20220924-46-1lsrm2c.pdf
 C15.gdoc
'car rent.gdoc'
' Ceasar cipher ,Rail fence cipher.gdoc'
 Classroom
'CNS UNIT 1.gdoc'
'CN  Units 1,2 (1).gdoc'
'CN  Units 1,2.gdoc'
'Colab Notebooks'
'Data mining 115,117 (1).docx'
'Data mining 115,117.docx'
'db fire base.gdoc'
 exp1.gdoc
'EXP_NO_1_cns (1).docx'
 EXP_NO_1_cns.docx
'FOMC abstract.gdoc'
'ICT Conference Physical Attendees List.xlsx'
 IMG_20220902_213352__01.jpg
 IMG_20221005_231541_171.jpg
 Iptables.gdoc
'IT17711 Data Analytics Lab Record.gdoc'
 javas

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15502, done.
remote: Total 15502 (delta 0), reused 0 (delta 0), pack-reused 15502
Receiving objects: 100% (15502/15502), 14.17 MiB | 22.09 MiB/s, done.
Resolving deltas: 100% (10403/10403), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
  946 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
 1147 |             if (iteration_

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "rajesh" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2023-02-22 07:46:17--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  62.9MB/s    in 2.5s    

2023-02-22 07:46:20 (62.9 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/classes.txt    
  inflating: data/obj/IMG_20201216_233914.jpg  
  inflating: data/obj/IMG_20201216_233914.txt  
  inflating: data/obj/IMG_20210305_175730_961.jpg  
  inflating: data/obj/IMG_20210305_175730_961.txt  
  inflating: data/obj/IMG_20210317_201034.jpg  
  inflating: data/obj/IMG_20210317_201034.txt  
  inflating: data/obj/IMG_20210317_201534.jpg  
  inflating: data/obj/IMG_20210317_201534.txt  
  inflating: data/obj/IMG_20210318_102750.jpg  
  inflating: data/obj/IMG_20210318_102750.txt  
  inflating: data/obj/IMG_20210401_133205.jpg  
  inflating: data/obj/IMG_20210401_133205.txt  
  inflating: data/obj/IMG_20210411_224151.jpg  
  inflating: data/obj/IMG_20210411_224151.txt  
  inflating: data/obj/IMG_20210418_094255.jpg  
  inflating: data/obj/IMG_20210418_094255.txt  
  inflating: data/obj/IMG_20210426_070819.jpg  
  inflating: data/obj/IMG_20210426_070819.txt  
  inflating: data/obj/IMG_20210611_135354.jpg  
  inf

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.447188 0.536250 0.224375 0.452500

['15', '0.447188', '0.536250', '0.224375', '0.452500']
0 data/obj/IMG-20230118-WA0010.txt
0 0.447188 0.536250 0.224375 0.452500
15 0.711096 0.546562 0.561570 0.313125

['15', '0.711096', '0.546562', '0.561570', '0.313125']
1 data/obj/IMG_20220206_070357.txt
0 0.711096 0.546562 0.561570 0.313125
15 0.633437 0.578333 0.176875 0.326667

['15', '0.633437', '0.578333', '0.176875', '0.326667']
2 data/obj/IMG_20220428_123716.txt
0 0.633437 0.578333 0.176875 0.326667
15 0.531563 0.421250 0.239375 0.457500

['15', '0.531563', '0.421250', '0.239375', '0.457500']
3 data/obj/IMG-20230118-WA0006.txt
0 0.531563 0.421250 0.239375 0.457500
15 0.546875 0.443866 0.081597 0.142361

['15', '0.546875', '0.443866', '0.081597', '0.142361']
4 data/obj/IMG-20210920-WA0012.txt
0 0.546875 0.443866 0.081597 0.142361
15 0.501563 0.173828 0.198437 0.230469

['15', '0.501563', '0.173828', '0.198437', '0.230469']
5 data/obj/IMG-20210319-WA0014.txt
0 0.501563 0.173828 0.198437 0

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/IMG_20220413_110652.jpg', 'data/obj/IMG_20211202_202746.jpg', 'data/obj/IMG_20211109_203847.jpg', 'data/obj/IMG_20210620_112018.jpg', 'data/obj/IMG_20210305_175730_961.jpg', 'data/obj/IMG_20210731_155106.jpg', 'data/obj/IMG_20220413_121418.jpg', 'data/obj/IMG_20220916_163030.jpg', 'data/obj/IMG_20220731_212333.jpg', 'data/obj/IMG_20220101_000056.jpg', 'data/obj/IMG_20221204_185438.jpg', 'data/obj/IMG_20210620_112219.jpg', 'data/obj/IMG_20210812_214827.jpg', 'data/obj/IMG_20210317_201534.jpg', 'data/obj/IMG-20220315-WA0081.jpg', 'data/obj/IMG-20230118-WA0010.jpg', 'data/obj/IMG-20210301-WA0014.jpg', 'data/obj/IMG-20230118-WA0020.jpg', 'data/obj/IMG_20220413_121248.jpg', 'data/obj/IMG_20220718_221340.jpg', 'data/obj/IMG_20210318_102750.jpg', 'data/obj/IMG-20210328-WA0004.jpg', 'data/obj/IMG-20211031-WA0011.jpg', 'data/obj/IMG-20211209-WA0011.jpg', 'data/obj/IMG_20210317_201034.jpg', 'data/obj/IMG_20201216_233914.jpg', 'data/obj/IMG_20220317_163836.jpg', 'data/obj/IMG_20230102_

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 248077, rewritten_bbox = 0.000000 % 
]2;3926/4000: loss=0.0 hours left=0.3
 3926: 0.016917, 0.027169 avg loss, 0.001000 rate, 8.258615 seconds, 251264 images, 0.263579 hours left
Loaded: 0.000063 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.911142), count: 4, class_loss = 0.000023, iou_loss = 0.023347, total_loss = 0.023370 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 248081, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.907679), count: 4, class_loss = 0.000023, iou_loss = 0.024628, total_loss = 0.024651 
v3 (m